Acknowledgement:
    1. The structure of the code is inspired by depmixS4: An R Package for Hidden Markov Models: https://cran.r-project.org/web/packages/depmixS4/vignettes/depmixS4.pdf
    2. Some of the linear model codes are adapted from sklearn: http://scikit-learn.org/stable/ and statsmodel: http://statsmodels.sourceforge.net/. Some modifications have been made to these codes to realize more functionalities.

Problems with existing packages
    1. Some of sklearn and statsmodels does not support the implementation of sample weights
    2. Some of sklearn and statsmodels does not support l1, l2 or elasticnet regularizations
    3. Sklearn packages does not support estimation of standard deviation of coefficients
    4. The likelihood function of weighted linear models is not the same as the ones we need to use in IO-HMM
    5. In the R package aformentioned, they do not support the provision of multiple sequences.

Modifications to above packages:
    1. Implemented supervised models that supports sample weights
    2. Supports the estimation of standard deviations of coefficients
    3. Supports multiple regularizations (l1, l2, elastic net) in most of the supervised models. (However,  if regularization is applied, no standard deviation of the coefficients will be estimated)
    4. Supports estimation over multiple sequences (multiple dataframes)
    5. HMM forward-backward code was implemented at the log scale so that it is more robust to long sequences.
    6. Supports generalized linear models with different link functions, just as statsmodel.

# Example use of UnSupervised_IOHMM

In [1]:
from __future__ import  division
import numpy as np
from copy import deepcopy
import sys
sys.path.append('./main')
sys.path.append('./auxiliary')
from IOHMM import UnSupervisedIOHMM, UnSupervisedIOHMMMapReduce
from SupervisedModels import LM, MNLP, MNLD
from scipy.misc import logsumexp
import pandas as pd
import warnings
warnings.simplefilter("ignore")

## Speed data - example 1

In [2]:
speed = pd.read_csv('data/speed.csv')
speed.head()

,Unnamed: 0,rt,corr,Pacc,prev
0,1,6.456770,cor,0.0,inc
1,2,5.602119,cor,0.0,cor
2,3,6.253829,inc,0.0,cor
3,4,5.451038,inc,0.0,inc
4,5,5.872118,inc,0.0,inc


## Setting up the model

In [3]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-2)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData([speed])

In [4]:
SHMM.train()

-305.269386556
-305.233834447
-305.161662823
-305.048268232
-304.832808913
-304.32300925
-303.166779811
-298.430474021
-275.969175281
-193.133503589
-103.483023892
-93.2321920797
-92.6165992936
-92.6114240953


## See the coefficients

In [5]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.90956233  0.09043767]]
[[ 0.20087668  0.79912332]]


In [6]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef

[ 6.37769645]
[ 5.49908326]


In [7]:
print np.sqrt(SHMM.model_emissions[0][0].dispersion)
print np.sqrt(SHMM.model_emissions[1][0].dispersion)

0.248940948254
0.179536535128


## Speed data - example 2

In [8]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-2)
SHMM.setModels(model_emissions = [LM(est_sd = True), MNLD(est_sd=True)], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[],['Pacc']])
SHMM.setOutputs([['rt'],['corr']])
SHMM.setData([speed])

In [9]:
SHMM.train()

-527.845221767
-510.139309385
-423.049315621
-328.291919866
-307.806235992
-302.562299581
-303.974774219
-303.553705003
-303.303337084
-303.125041067
-302.993736176
-302.897539499
-302.827513121
-302.776742066
-302.739968965
-302.713291297
-302.693871586
-302.679673228
-302.669243788
-302.661548336


In [10]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.90700953  0.09299047]]
[[ 0.199721  0.800279]]


In [11]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[0][1].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[1][1].coef

[ 6.38146063]
[[ 0.         -0.99721026]
 [ 0.         -2.42263093]]
[ 5.50374218]
[[ 0.         -0.22046136]
 [ 0.          0.62761743]]


In [12]:
print SHMM.model_emissions[0][0].sd
print SHMM.model_emissions[0][1].sd
print SHMM.model_emissions[1][0].sd
print SHMM.model_emissions[1][1].sd

[ 0.01508935]
[[ 0.          0.37317603]
 [ 0.          0.79633655]]
[ 0.01364248]
[[ 0.          0.15889045]
 [ 0.          0.73583272]]


## MapReduce Version

In [13]:
sc.stop()
sc = SparkContext(appName="", pyFiles=[
    './auxiliary/HMM.py',
    './auxiliary/SupervisedModels.py',
    './auxiliary/family.py',
    './main/IOHMM.py'])

In [14]:
speed = pd.read_csv('data/speed.csv', index_col=0)
indexes = [(1,1), (2,1)]
RDD = sc.parallelize(indexes)
dfs = RDD.mapValues(lambda v: speed)

In [15]:
SHMM = UnSupervisedIOHMMMapReduce(num_states=2, max_EM_iter=100, EM_tol=1e-4)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData(dfs)

In [16]:
SHMM.train()
print 'done'

-610.62437275
-610.617904655
-610.611911166
-610.598565436
-610.584909147
-610.561227296
-610.520350122
-610.458884389
-610.404975956
-610.307587422
-610.104129263
-609.850277538
-609.156357816
-607.783070117
-601.741611831
-548.448524976
-348.589171936
-201.792161311
-182.927044571
-179.347994159
-178.307715154
-177.870366352
-177.656188106
-177.568272248
-177.536074621
-177.524155413
-177.519529985
-177.51760449
-177.516738529
-177.516321147
-177.516109039
-177.515997299
-177.515937088
done


In [17]:
sc.stop()